In [62]:
from json import loads, dumps
from requests import get
from types import SimpleNamespace
from dataclasses import dataclass, field

In [125]:
def load_config(filename='config.json'):
    with open(filename, 'r') as file:
        return SimpleNamespace(**loads(file.read()))
    
def download_attack(url):
    response = get(url)
    if response.ok:
        return response.json()
    
def collect_objects(attack):
    collections = {}
    for obj in attack['objects']:
        tp = obj['type']
        if tp in collections:
            collections[tp].append(obj)
        else:
            collections.update({
                tp: [obj]
            })
    return collections

def get_technique_id(references):
    for ref in references:
        if ref['source_name'] == 'mitre-attack':
            return ref['external_id']
    return None

@dataclass
class ATechnique:
    name: str
    description: str = ""
    detection: str = ""
    data_sources: list = field(default_factory=list)
    platforms: list = field(default_factory=list)
    permissions_required: list = field(default_factory=list)
    defenses_bypassed: list = field(default_factory=list)
    id: str = ""
    tactics: list = field(default_factory=list)
    references: list = field(default_factory=list)
    ref: str = ""
    
@dataclass
class ARelation:
    source : str
    target : str
    description: str
    type: str
    ref: str
    
def parse_techniques(objects):
    techniques = []
    for obj in collections['attack-pattern']:

        if 'revoked' in obj and obj['revoked']: continue

        id = get_technique_id(obj['external_references'])
        
        tactics = []
        for tac in obj['kill_chain_phases']:
            if tac['kill_chain_name'] == 'mitre-attack':
                tactics.append(
                    tac['phase_name']
                )

        defense = []
        if 'x_mitre_defense_bypassed' in obj and obj['x_mitre_defense_bypassed']:
            defense = obj['x_mitre_defense_bypassed']

        permissions = []
        if 'x_mitre_permissions_required' in obj and obj['x_mitre_permissions_required']:
            permissions = obj['x_mitre_permissions_required']

        tech = {
            'name': obj['name'],
            'platforms': obj['x_mitre_platforms'],
            'permissions_required': permissions,
            'defenses_bypassed': defense,
            'description': obj['description'],
            'detection': obj['x_mitre_detection'],
            'tactics': tactics,
            'id': id,
            'ref': obj['id']
        }

        atech = ATechnique(**tech)
        techniques.append(atech)
    return techniques

def parse_relations(relations):
    result_relations = []
    for rel in relations:
        data = {
            'source' : rel['source_ref'],
            'target' : rel['target_ref'],
            'description': rel.get('description', ''),
            'type': rel['relationship_type'],
            'ref': rel['id']
        }
        result_relations.append(
            ARelation(**data)
        )
    return result_relations


In [100]:
cfg = load_config()

In [101]:
attack = download_attack(cfg.attack_data)

In [102]:
collections = collect_objects(attack)

In [118]:
techniques = parse_techniques(collections['attack-pattern'])

In [126]:
relations = parse_relations(collections['relationship'])